In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

ft_stats = spark.read.parquet(
    "/Volumes/workspace/default/pokemon/gold/pokemon_ft_stats.parquet"
)
dim_type = spark.read.parquet(
    "/Volumes/workspace/default/pokemon/gold/pokemon_dim_type.parquet"
)
dim_abilities = spark.read.parquet(
    "/Volumes/workspace/default/pokemon/gold/pokemon_dim_abilities.parquet"
)

df_stats = ft_stats.toPandas()
df_type = dim_type.toPandas()
df_abilities = dim_abilities.toPandas()

Quantidade de Pokemons por tipo

In [0]:
count_tipo1 = df_type['tipo_1'].value_counts().to_frame().reset_index()
count_tipo2 = df_type['tipo_2'].value_counts().to_frame().reset_index()
single_type_count = df_type['tipo_2'].isna().sum()
multi_type_count = df_type['tipo_2'].notna().sum()

plt.figure(figsize=(10, 6))
sns.barplot(x='index', y='tipo_1', data=count_tipo1)
plt.title('Contagem de Pokémon por Tipo 1')
plt.xlabel('Tipo 1')
plt.ylabel('Contagem')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='index', y='tipo_2', data=count_tipo2)
plt.title('Contagem de Pokémon por Tipo 2')
plt.xlabel('Tipo 2')
plt.ylabel('Contagem')
plt.xticks(rotation=45)
plt.show()

labels = ["1 Tipo", "2 Tipos"]
values = [single_type_count, multi_type_count]

plt.figure(figsize=(6,6))
plt.pie(values, labels=labels, autopct="%1.1f%%", startangle=90)
plt.title("Pokémons com 1 ou 2 tipos")
plt.show()

In [0]:
df_stats = pd.DataFrame(df_stats)

df_stats["total_stats"] = df_stats[["attack","defense","hp","special_attack","special_defense","speed"]].sum(axis=1)
top_total = df_stats.nlargest(10, "total_stats")

plt.figure(figsize=(10,6))
plt.bar(top_total["name"], top_total["total_stats"], color="royalblue")
plt.title("Top 10 Pokémons - Stats Acumulados")
plt.ylabel("Total de Stats")
plt.xticks(rotation=45)
plt.show()

stats = ["attack","defense","hp","special_attack","special_defense","speed"]

for stat in stats:
    top_stat = df_stats.nlargest(10, stat)

    plt.figure(figsize=(10,6))
    plt.bar(top_stat["name"], top_stat[stat], color="orange")
    plt.title(f"Top 10 Pokémons - {stat.capitalize()}")
    plt.ylabel(stat.capitalize())
    plt.xticks(rotation=45)
    plt.show()

In [0]:
def to_na(x):
    return None if (pd.isna(x) or str(x).strip().lower() in {"null", "none", "nan", ""}) else x

cols = ["habilidade_1","habilidade_2","habilidade_3"]
for c in cols:
    df_abilities[c] = df_abilities[c].map(to_na)


df_abilities["qtd_habilidades"] = df_abilities[cols].apply(lambda r: r.notna().sum(), axis=1)

vc = df_abilities["qtd_habilidades"].value_counts()
resumo = pd.DataFrame([{
    "pokemons_com_1_habilidade": int(vc.get(1, 0)),
    "pokemons_com_2_habilidades": int(vc.get(2, 0)),
    "pokemons_com_3_habilidades": int(vc.get(3, 0)),
}])

resumo


vals = [
    resumo.loc[0,"pokemons_com_1_habilidade"],
    resumo.loc[0,"pokemons_com_2_habilidades"],
    resumo.loc[0,"pokemons_com_3_habilidades"],
]

plt.figure()
plt.bar(["1 habilidade","2 habilidades","3 habilidades"], vals)
plt.title("Pokémons por quantidade de habilidades")
plt.xlabel("Quantidade de habilidades")
plt.ylabel("Número de Pokémons")
plt.show()